In [3]:
from datetime import datetime
import h5py
import numpy as np
import matplotlib.pyplot as plt
import numba
from numba import cuda
import pandas as pd
import plotly
import plotly.graph_objects as go
import time
from matplotlib.cm import get_cmap
from scipy.spatial.distance import cdist
import os
import concurrent.futures
from matplotlib.cm import get_cmap
from tqdm import tqdm
from joblib import Parallel, delayed

plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams.update({'font.size': 12})
plt.rcParams.update({'font.style': 'oblique'})
print("hello")


# lat_river_mouth, lon_river_mouth = 41.139024, -8.680089
# figpath = "fig/"

hello


In [4]:

filepath = "raw/March/"
files = os.listdir(filepath)
files.sort()

print(files)

['salinity_mar_2016_1-009.mat', 'salinity_mar_2016_2-008.mat', 'salinity_mar_2016_3-011.mat', 'salinity_mar_2017_1-002.mat', 'salinity_mar_2017_2-010.mat', 'salinity_mar_2017_3-007.mat', 'salinity_mar_2018_1-004.mat', 'salinity_mar_2018_2-012.mat', 'salinity_mar_2018_3-006.mat', 'salinity_mar_2019_1-001.mat', 'salinity_mar_2019_2-005.mat', 'salinity_mar_2019_3-003.mat']


In [5]:
# s1: load wind and tide
path_wind = "wind/raw/wind_data.txt"
path_tide = "tide/raw/Month_11.csv"
wind = np.loadtxt(path_wind, delimiter=",")
tide = pd.read_csv(path_tide).to_numpy()
# tide = np.loadtxt(path_tide, delimiter=", ")

def angle2anle(angle):
    return 270 - angle

def s2uv(s, angle):
    angle = angle2anle(angle)
    u = s * np.cos(deg2rad(angle))
    v = s * np.sin(deg2rad(angle))
    return u, v

def rad2deg(rad):
    return rad / np.pi * 180

def deg2rad(deg):
    return deg / 180 * np.pi

def vectorize(v):
    return np.array(v).reshape(-1, 1)


In [6]:
def get_indices(timestamp):
    # s1: get wind index
    dist_wind = np.abs(timestamp - wind[:, 0])
    ind_wind = np.where(dist_wind == np.nanmin(dist_wind))[0][0]
    
    # s2: get tide index
    dist_tide = np.abs(timestamp - tide[:, 0])
    ind_tide = np.where(dist_tide == np.nanmin(dist_tide))[0][0]

    # s3: evaluate
    tide_string = "None"
    wind_dir = "None"
    wind_level = "None"
    if tide[ind_tide, 2] == 1:
        if timestamp >= tide[ind_tide, 0]:
            tide_string = "Ebb"
        else:
            tide_string = "Flood"
    else:
        if timestamp >= tide[ind_tide, 0]:
            tide_string = "Flood"
        else:
            tide_string = "Ebb"
    angles = np.arange(4) * 90 + 45
    directions = ['East', 'South', 'West', 'North']
    ind = len(angles[angles < wind[ind_wind, 2]]) - 1
    wind_dir = directions[ind]
    
    speeds = np.array([0, 2.5, 6])
    levels = ['Mild', 'Moderate', 'Heavy']
    ind = len(speeds[speeds < wind[ind_wind, 1]]) - 1
    wind_level = levels[ind]
    
#     print("wind: ", wind_dir)
#     print("wind: ", wind_level)
#     print("tide: ", tide_string)
    return timestamp, wind_dir, wind_level, tide_string


for file in files:
    print(file)

    date_string = file[-14:-8]  #TODO:  Just for March, change for specific month 
    data = h5py.File(filepath + file, 'r')
    t1 = time.time()
    df = data.get('data')
    Time = np.array(df.get('Time'))
    timestamp = (Time - 719529) * 24 * 3600  # 719529 is how many days have passed from Jan1 0,
    t2 = time.time()
    
    timestamp_data = timestamp[0]
#     timestamp = timestamp[0]
#     get_indices(timestamp[0])
    
#     try:
    wd = []
    wl = []
    ts = []
    tp = []
    for t in timestamp_data:
        t1, w1, w2, t2 = get_indices(t)
        tp.append(t1)
        wd.append(w1)
        wl.append(w2)
        ts.append(t2)
#     res = Parallel(n_jobs=2)(delayed(get_indices)(t) for t in timestamp_data)
#     tp = [item[0] for item in res]
#     wind_dir = [item[1] for item in res]
#     wind_level = [item[2] for item in res]
#     tide_string = [item[3] for item in res]

#     dataf = {"timestamp": tp, "wind_direction": wind_dir, "wind_level": wind_level, "tide": tide_string}
    dataf = {"timestamp": tp, "wind_direction": wd, "wind_level": wl, "tide": ts}
    dataset = pd.DataFrame(dataf)
    dataset.to_csv("indices/F_"+date_string+".csv", index=False)
#     except:
#         pass


salinity_mar_2016_1-009.mat
salinity_mar_2016_2-008.mat
salinity_mar_2016_3-011.mat
salinity_mar_2017_1-002.mat
salinity_mar_2017_2-010.mat
salinity_mar_2017_3-007.mat
salinity_mar_2018_1-004.mat
salinity_mar_2018_2-012.mat
salinity_mar_2018_3-006.mat
salinity_mar_2019_1-001.mat
salinity_mar_2019_2-005.mat
salinity_mar_2019_3-003.mat


In [13]:
file[-14:-8]

'2016_1'

In [5]:
file = files[0]

In [9]:
file

'salinity_mar_2016_1-009.mat'

In [1]:
dataf

NameError: name 'dataf' is not defined